XGBoost benchmark

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
DATA_FOLDER = '../input'
transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
test            = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))

In [ ]:
train_df = test
train_df['item_category_id'] = train_df["item_id"].map(items['item_category_id'])

In [ ]:
for x in range(34):
    target_df = transactions[transactions['date_block_num'] == x].groupby(['shop_id', 'item_id'], as_index=False).item_cnt_day.sum().rename(columns={"item_cnt_day": x})
    train_df = train_df.merge(target_df, how="left", on=['shop_id', 'item_id']).fillna(0.0)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(train_df.sum()[4:])
plt.axvline(x=33, color='r')
plt.axvline(x=33-9, color='g')
plt.axvline(x=33-12, color='r')
plt.axvline(x=33-21, color='g')
plt.axvline(x=33-24, color='r')
plt.axvline(x=0, color='g')

we can clearly see some seasonality here, the green line is START of training set, red line is END of training set. The prediction is red + 1.
Let's modify the df accordingly

In [ ]:
train_df = test
train_df['item_category_id'] = train_df["item_id"].map(items['item_category_id'])

In [ ]:
windowsize = 9
k = 12
for x in range(k, k + windowsize + 2):
    target_df = transactions[transactions['date_block_num'] == x].groupby(['shop_id', 'item_id'], as_index=False).item_cnt_day.sum().rename(columns={"item_cnt_day": x - k})
    train_df = train_df.merge(target_df, how="left", on=['shop_id', 'item_id']).fillna(0.0)

In [ ]:
train_df.head()

In [ ]:
plt.plot(train_df.sum()[4:])
plt.axvline(x=9, color='r')
plt.axvline(x=0, color='g')

In [ ]:
train_df2 = test
train_df2['item_category_id'] = train_df2["item_id"].map(items['item_category_id'])

In [ ]:
windowsize = 9
k = 0
for x in range(k, k + windowsize + 2):
    target_df = transactions[transactions['date_block_num'] == x].groupby(['shop_id', 'item_id'], as_index=False).item_cnt_day.sum().rename(columns={"item_cnt_day": x - k})
    train_df2 = train_df2.merge(target_df, how="left", on=['shop_id', 'item_id']).fillna(0.0)

In [ ]:
train_df2.head()

In [ ]:
plt.plot(train_df2.sum()[4:])
plt.axvline(x=9, color='r')
plt.axvline(x=0, color='g')

In [ ]:
train_df = pd.concat([train_df, train_df2])

here we isolated the period previous to the prediction

<h1>Modelling

In [ ]:
import xgboost as xgb

In [ ]:
xgbtrain = xgb.DMatrix(train_df.iloc[:, (train_df.columns != 'ID') & (train_df.columns != 10)].values, train_df.iloc[:, (train_df.columns == 10)].values)

In [ ]:
param = {'max_depth':10, 
         'subsample':1,
         'min_child_weight':0.5,
         'eta':0.3, 
         'num_round':1000, 
         'seed':1,
         'silent':0,
         'eval_metric':'rmse'}

In [ ]:
bst = xgb.train(param, xgbtrain)

In [ ]:
xgb.plot_importance(bst)

<h1>Predictions

In [ ]:
predict_df = test
predict_df['item_category_id'] = predict_df["item_id"].map(items['item_category_id'])

In [ ]:
windowsize = 9
k = 24
for x in range(k, k + windowsize + 1):
    target_df = transactions[transactions['date_block_num'] == x].groupby(['shop_id', 'item_id'], as_index=False).item_cnt_day.sum().rename(columns={"item_cnt_day": x - k})
    predict_df = predict_df.merge(target_df, how="left", on=['shop_id', 'item_id']).fillna(0.0)

In [ ]:
plt.plot(predict_df.sum()[4:])
plt.axvline(x=9, color='r')
plt.axvline(x=0, color='g')

In [ ]:
predict_df.head()

In [ ]:
xgbpredict = xgb.DMatrix(predict_df.iloc[: , (predict_df.columns != 'ID')].values)

In [ ]:
pred = bst.predict(xgbpredict)

In [ ]:
plt.plot(np.append(np.array(predict_df.sum()[4:]), pred.sum()))
plt.axvline(x=9, color='r')
plt.axvline(x=0, color='g')

In [ ]:
pred.sum()

In [ ]:
pred = pred.clip(0, 20)

In [ ]:
sub_df = pd.DataFrame({'ID':predict_df.ID,'item_cnt_month': pred })

In [ ]:
sub_df.describe()

In [ ]:
sub_df.to_csv('xg_boost4_cats.csv',index=False)